# BIBLIOTECA

In [4]:
pip install certifi

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip-system-certs


Usage:   
  /home/codespace/.python/current/bin/python -m pip <command> [options]

no such option: -s
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import ssl
import requests
import certifi
from io import StringIO
import datetime
import time

# WEB SCRAPING BCB ORIGINAL

In [5]:
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,id,dataMovimento,dataLiquidacao,edital,tipoPublico,prazo,quantidadeOfertada,quantidadeAceita,codigoTitulo,dataVencimento,tipoOferta,ofertante,quantidadeOfertadaSegundaRodada,quantidadeAceitaSegundaRodada,cotacaoMedia,cotacaoCorte,taxaMedia,taxaCorte,financeiro
0,9fe038043290d953509e7c28000155b,2025-02-27 00:00:00,2025-02-28 00:00:00,48,TodoMercado,2133,500000,500000,950199,2031-01-01 00:00:00,Venda,Tesouro Nacional,125000.0,0.0,838775172,83869127,149713,14974,4194
1,9fe038043290d953509e7c28000155a,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,2498,1000000,1000000,100000,2032-01-01 00:00:00,Venda,Tesouro Nacional,250000.0,0.0,388886769,388563686,149149,14929,3889
2,9fe038043290d953509e7c280001559,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,1403,2000000,2000000,100000,2029-01-01 00:00:00,Venda,Tesouro Nacional,500000.0,0.0,591579107,591506878,147913,14795,11832
3,9fe038043290d953509e7c280001558,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,762,2000000,2000000,100000,2027-04-01 00:00:00,Venda,Tesouro Nacional,500000.0,0.0,753121088,753121088,14729,14729,15062
4,9fe038043290d953509e7c280001557,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,397,1000000,1000000,100000,2026-04-01 00:00:00,Venda,Tesouro Nacional,250000.0,0.0,861370251,861348997,148271,148298,8614


# SELIC - RESULTADOS DOS LEILÕES

In [1]:
import ssl
import pandas as pd
from datetime import datetime

ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# 1. Reorganizar as colunas
colunas_desejadas = ['edital', 'tipoOferta', 'codigoTitulo', 'ofertante', 'prazo', 'dataLiquidacao','quantidadeOfertada', 'quantidadeAceita', 'taxaCorte']
tabela = tabela[colunas_desejadas]

# 2. Criar a coluna "NomeTitulo"
tabela['NomeTitulo'] = tabela['codigoTitulo'].map({210100: 'LFT 210100', 760199: 'NTN-B 760199'})

# 3. Mover a coluna "NomeTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('codigoTitulo') + 1, colunas.pop(colunas.index('NomeTitulo')))
tabela = tabela[colunas]

# 4. Ordenar por 'edital' com ordem personalizada
ordem_edital = [37, 38, 39]
tabela['edital'] = pd.Categorical(tabela['edital'], categories=ordem_edital, ordered=True)

# 5. Ordenar por 'prazo' com ordem personalizada
ordem_prazo = sorted(tabela['prazo'].unique())
tabela['prazo'] = pd.Categorical(tabela['prazo'], categories=ordem_prazo, ordered=True)

# 6. Criar a coluna "AnoTitulo"
tabela['AnoTitulo'] = (datetime.now() + pd.to_timedelta(pd.to_numeric(tabela['prazo'], errors='coerce'), unit='D')).dt.year

# 7. Mover a coluna "AnoTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('NomeTitulo') + 1, colunas.pop(colunas.index('AnoTitulo')))
tabela = tabela[colunas]

# 8. Formatar a coluna "taxaCorte" como porcentagem
tabela['taxaCorte'] = pd.to_numeric(tabela['taxaCorte'], errors='coerce') / 10000
tabela['taxaCorte'] = tabela['taxaCorte'].astype(str) + '%'

# 9. Formatar a coluna "quantidadeAceita"
tabela['quantidadeAceita'] = tabela['quantidadeAceita'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))


# 10. Formatar a coluna "quantidadeOfertada"
tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,edital,tipoOferta,codigoTitulo,NomeTitulo,AnoTitulo,ofertante,prazo,dataLiquidacao,quantidadeOfertada,quantidadeAceita,taxaCorte
0,NaN,Venda,950199,NaN,2031,Tesouro Nacional,2133,2025-02-28 00:00:00,500.000,500.000,1.4974%
1,NaN,Venda,100000,NaN,2032,Tesouro Nacional,2498,2025-02-28 00:00:00,1.000.000,1.000.000,1.4929%
2,NaN,Venda,100000,NaN,2029,Tesouro Nacional,1403,2025-02-28 00:00:00,2.000.000,2.000.000,1.4795%
3,NaN,Venda,100000,NaN,2027,Tesouro Nacional,762,2025-02-28 00:00:00,2.000.000,2.000.000,1.4729%
4,NaN,Venda,100000,NaN,2026,Tesouro Nacional,397,2025-02-28 00:00:00,1.000.000,1.000.000,14.8298%
